## Сборный проект-2: Удовлетворенность сотрудников

### Описание проекта 

HR-аналитики компании «Работа с заботой» помогают бизнесу оптимизировать управление персоналом: бизнес предоставляет данные, а аналитики предлагают, как избежать финансовых потерь и оттока сотрудников. В этом HR-аналитикам пригодится машинное обучение, с помощью которого получится быстрее и точнее отвечать на вопросы бизнеса.

Компания предоставила данные с характеристиками сотрудников компании. Среди них — уровень удовлетворённости сотрудника работой в компании. Эту информацию получили из форм обратной связи: сотрудники заполняют тест-опросник, и по его результатам рассчитывается доля их удовлетворённости от 0 до 1, где 0 — совершенно неудовлетворён, 1 — полностью удовлетворён. 

Собирать данные такими опросниками не так легко: компания большая, и всех сотрудников надо сначала оповестить об опросе, а затем проследить, что все его прошли. 



#### Задачи 

1) Первая — построить модель, которая сможет предсказать уровень удовлетворённости сотрудника на основе данных заказчика. 

Почему бизнесу это важно: удовлетворённость работой напрямую влияет на отток сотрудников. А предсказание оттока — одна из важнейших задач HR-аналитиков. 

Внезапные увольнения несут в себе риски для компании, особенно если уходит важный сотрудник.


2) Ваша вторая задача — построить модель, которая сможет на основе данных заказчика предсказать то, что сотрудник уволится из компании.

### Нужные библиотеки и загрузка данных

In [2]:
# библиотеки для работы с данными
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math

# библиотека для оценки важности признаков Шепли (SHAP)
import shap

# бибилотеки для подготовки данных
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import (StandardScaler, 
                                   MinMaxScaler, 
                                   RobustScaler,
                                   PolynomialFeatures, 
                                   OneHotEncoder,
                                   OrdinalEncoder, 
                                   LabelEncoder)

from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE


# библиотеки для пайплайнов
# класс ColumnTransformer помогает работать с данными разного типа в одном наборе
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


# класс для работы с пропусками
from sklearn.impute import SimpleImputer

# библиотеки для линейной регрессии
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error ,mean_absolute_error, r2_score


# библиотеки для логистической регрессии
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay

# библиотека для метода опорных векторов
from sklearn.svm import SVC

# библиотека решающих деревьев
from sklearn.tree import DecisionTreeClassifier


# бибилиотека для графика ROC-кривой и расчёта ROC-AUC
from sklearn.metrics import roc_auc_score, RocCurveDisplay


# библиотека для модели классификации kNN 
from sklearn.neighbors import KNeighborsClassifier

# добавим библиотеку для построения scatter диаграмм рассеяния с тренд-линиями
import plotly.express as px 
import plotly.graph_objects as go



Читаем исходные датасеты

In [3]:
try:
    train_job_satisfaction_rate_df = pd.read_csv('/datasets/train_job_satisfaction_rate.csv')
    train_quit_df = pd.read_csv('/datasets/train_quit.csv')
    test_features_df = pd.read_csv('/datasets/test_features.csv')
    test_target_job_satisfaction_rate_df= pd.read_csv('/datasets/test_target_job_satisfaction_rate.csv')#, sep=';',decimal=',')
    test_target_quit_df = pd.read_csv('/datasets/test_target_quit.csv')

except FileNotFoundError:
    train_job_satisfaction_rate_df = pd.read_csv('/Users/tony_builder/Documents/GitHub/studying/yandex_practicum/data/train_job_satisfaction_rate.csv')
    train_quit_df = pd.read_csv('/Users/tony_builder/Documents/GitHub/studying/yandex_practicum/data/train_quit.csv')
    test_features_df = pd.read_csv('/Users/tony_builder/Documents/GitHub/studying/yandex_practicum/data/test_features.csv')
    test_target_job_satisfaction_rate_df= pd.read_csv('/Users/tony_builder/Documents/GitHub/studying/yandex_practicum/data/test_target_job_satisfaction_rate.csv')#, sep=';',decimal=',')
    test_target_quit_df = pd.read_csv('/Users/tony_builder/Documents/GitHub/studying/yandex_practicum/data/test_target_quit.csv')


In [4]:
# функция получения имени датасета

def get_dataset_name(variable):
    import inspect
    frame = inspect.currentframe()
    for scope in [frame.f_back.f_globals, frame.f_back.f_locals]:
        for var_name, var_val in scope.items():
            if isinstance(var_val, pd.DataFrame) and var_val is variable:
                del frame
                return var_name


In [5]:
# функция выбора цвета для текста output 

from IPython.display import Markdown, display

def print_red(text):
    display(Markdown(f'<span style="color:red">{text}</span>'))

Выводим информацию об исходных датасетах

In [6]:
for df in [train_job_satisfaction_rate_df, train_quit_df, test_features_df, test_target_job_satisfaction_rate_df, test_target_quit_df]:
    import inspect
    caller_locals = inspect.currentframe().f_back.f_locals
    dataset_name = get_dataset_name(df)
    # pd.set_option('display.max_rows', None)

    print_red(f'Информация по таблице {dataset_name}')
    display(df.head())
    print()
    display(df.info())
    print()
    

<span style="color:red">Информация по таблице train_job_satisfaction_rate_df</span>

,id,dept,level,workload,employment_years,last_year_promo,last_year_violations,supervisor_evaluation,salary,job_satisfaction_rate
0,155278,sales,junior,medium,2,no,no,1,24000,0.58
1,653870,hr,junior,high,2,no,no,5,38400,0.76
2,184592,sales,junior,low,1,no,no,2,12000,0.11
3,171431,technology,junior,low,4,no,no,2,18000,0.37
4,693419,hr,junior,medium,1,no,no,3,22800,0.20



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     4000 non-null   int64  
 1   dept                   3994 non-null   object 
 2   level                  3996 non-null   object 
 3   workload               4000 non-null   object 
 4   employment_years       4000 non-null   int64  
 5   last_year_promo        4000 non-null   object 
 6   last_year_violations   4000 non-null   object 
 7   supervisor_evaluation  4000 non-null   int64  
 8   salary                 4000 non-null   int64  
 9   job_satisfaction_rate  4000 non-null   float64
dtypes: float64(1), int64(4), object(5)
memory usage: 312.6+ KB


None

<span style="color:red">Информация по таблице train_quit_df</span>

...

None

<span style="color:red">Информация по таблице test_target_quit_df</span>

,id,quit
0,999029,yes
1,372846,no
2,726767,no
3,490105,no
4,416898,yes



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2000 non-null   int64 
 1   quit    2000 non-null   object
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


None

### Задача 1: предсказание уровня удовлетворённости сотрудника

#### Шаг 1-2 Загрузка и предобработка данных

Признаки от заказчика:

        id — уникальный идентификатор сотрудника;
        dept — отдел, в котором работает сотрудник;
        level — уровень занимаемой должности;
        workload — уровень загруженности сотрудника;
        employment_years — длительность работы в компании (в годах);
        last_year_promo — показывает, было ли повышение за последний год;
        last_year_violations — показывает, нарушал ли сотрудник трудовой договор за последний год;
        supervisor_evaluation — оценка качества работы сотрудника, которую дал руководитель;
        salary — ежемесячная зарплата сотрудника;
        job_satisfaction_rate — уровень удовлетворённости сотрудника работой в компании, целевой признак.

В таблице train_job_satisfaction_rate_df есть пропуски в столбцах dept и level. выведем эти значения

In [7]:
train_job_satisfaction_rate_df.query('dept != dept | level != level')

,id,dept,level,workload,employment_years,last_year_promo,last_year_violations,supervisor_evaluation,salary,job_satisfaction_rate
1209,631073,sales,NaN,medium,1,no,no,4,27600,0.66
1469,416327,sales,NaN,low,1,no,no,5,18000,0.73
1526,694746,NaN,junior,medium,5,no,no,4,21600,0.62
1630,814624,NaN,junior,medium,3,no,no,4,24000,0.88
1633,475114,NaN,junior,high,4,no,no,4,31200,0.63
1745,135043,sales,NaN,medium,1,no,no,3,26400,0.30
2522,998838,sales,NaN,medium,1,no,no,5,27600,0.71
2781,497243,NaN,junior,medium,1,no,no,3,26400,0.28
2975,168668,NaN,junior,low,3,no,no,4,18000,0.88
3866,641150,NaN,junior,low,3,no,yes,4,12000,0.54


Информация ничего не дала, посмотрим какие департаменты и уровни вообще есть 

In [9]:
for feat in ['dept', 'level']:
    print(train_job_satisfaction_rate_df[feat].unique())

['sales' 'hr' 'technology' 'purchasing' 'marketing' nan]
['junior' 'middle' 'sinior' nan]


In [10]:
train_job_satisfaction_rate_df.groupby('level').agg({'salary': 'median'})

,salary
level,
junior,24000.0
middle,39600.0
sinior,56400.0


Можно построить распределения и boxplot, посмотреть пороговые значения для зарплат по уровням сотрудников и таким образом принять решение о заполнении пропусков.

Аналогичную выборку можно провести по департаментам уже после заполнения уровней (группировать по департаменту и уровню сотрудника) - и тоже заполнить пропуски.

Но с другой стороны - пустых значений всего 10 из 4000 - это 0,25% - просто удалим их

In [13]:
train_job_satisfaction_rate_df.dropna(inplace=True)

In [14]:
train_job_satisfaction_rate_df.isna().sum()

id                       0
dept                     0
level                    0
workload                 0
employment_years         0
last_year_promo          0
last_year_violations     0
supervisor_evaluation    0
salary                   0
job_satisfaction_rate    0
dtype: int64

In [15]:
train_job_satisfaction_rate_df.shape

(3990, 10)

Проверим на дубликаты

In [12]:
train_job_satisfaction_rate_df.duplicated().sum()

0

Дубликатов нет, посмотрим на неявные дубликаты/ опечатки

In [17]:
for feat in ['dept', 'level', 'workload']:
    print(train_job_satisfaction_rate_df[feat].unique())

['sales' 'hr' 'technology' 'purchasing' 'marketing']
['junior' 'middle' 'sinior']
['medium' 'high' 'low']


Никаких дубликатов в таблице нет.

Смотрим на таблицу test_features.csv

In [19]:
test_features_df.duplicated().sum()

0

In [21]:
for feat in ['dept', 'level', 'workload']:
    print(test_features_df[feat].unique())

['marketing' 'hr' 'sales' 'purchasing' 'technology' nan ' ']
['junior' 'middle' 'sinior' nan]
['medium' 'low' 'high' ' ']


неявных дубликатов нет, но есть и пропуски, и пустые значения , выведем эти строки

In [29]:
test_features_df[
                (test_features_df.dept.isna() )| 
                  (test_features_df.dept == ' ' ) |
                  (test_features_df.workload == ' ' )|
                  (test_features_df.level.isna())
                  ]

,id,dept,level,workload,employment_years,last_year_promo,last_year_violations,supervisor_evaluation,salary
15,590867,marketing,junior,,4,no,no,4,28800
191,609865,NaN,junior,medium,1,no,no,1,20400
312,471990,sales,NaN,low,1,no,no,3,12000
1196,832342,NaN,junior,medium,1,no,no,4,28800
1699,822186,,middle,medium,3,no,no,4,38400


аналогично, пропусков и пустых значений всего 0.25% - пренебрегаем ими, удалим

In [43]:
test_features_df.dropna(inplace=True)
test_features_df = test_features_df[
                  (test_features_df.dept != ' ' ) &
                  (test_features_df.workload != ' ' )
                  ]

In [44]:
test_features_df[
                (test_features_df.dept.isna() )| 
                  (test_features_df.dept == ' ' ) |
                  (test_features_df.workload == ' ' )|
                  (test_features_df.level.isna())
                  ]

,id,dept,level,workload,employment_years,last_year_promo,last_year_violations,supervisor_evaluation,salary


In [45]:
test_features_df.shape

(1995, 9)

смотрим таблицу test_target_job_satisfaction_rate.csv - выше оценили, с ней проблем нет